In [1]:
!pip install -r requirements.txt

/usr/lib/python3/dist-packages/secretstorage/dhcrypto.py:15: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/usr/lib/python3/dist-packages/secretstorage/util.py:19: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [4]:
from PARSER import PDF_PARSER

In [ ]:
def load_pdfs(path):
    return os.listdir(path)

In [ ]:
def BASIC_INFO(path):
    start_keyTerm = '''Quality, Currency, Availability of Textbooks and Other Instructional Materials'''
    end_keyTerm = '''School Facility Conditions and Planned Improvements'''
    PDF =  pdfplumber.open(path)
    PAGES = PDF.pages
    
    SCHOOL         = ''
    DISTRICT       = ''
    CDS_CODE       = ''
    COLLECTED_DATE = ''
    TEXT = []
    start = ''
    end = ''
    for e, PAGE in enumerate(PAGES):   
        if e == 1:
            Table1 = PAGE.extract_tables()[0]
            for lst in Table1:
                if 'School Name' in lst:
                    ind = lst.index('School Name')
                    SCHOOL = lst[ind+1]
                if 'County-District-School (CDS) Code' in lst:
                    ind = lst.index('County-District-School (CDS) Code')
                    CDS_CODE = lst[ind+1]
            Table2 = PAGE.extract_tables()[1]
            for lst in Table2:
                if 'District Name' in lst:
                    ind = lst.index('District Name')
                    DISTRICT = lst[ind+1]
          
        if PAGE.search(start_keyTerm) != []:
            start = e
         
        if PAGE.search(end_keyTerm) != []:
            end = e
            break
        
    for i in range(start, end+1):
        TXT = PAGES[i].extract_text()    
        LST = TXT.split('\n')
        TEXT.extend(LST)
        TEXT.remove(TEXT[-1])
    limits = []
    for i in TEXT:
        if start_keyTerm in i:
            limits.append(TEXT.index(i))
        if end_keyTerm in i:
            limits.append(TEXT.index(i))
    TEXT = TEXT[limits[0]:limits[-1]]
    
    for i in TEXT:
        if 'Year and month in which the data were collected' in i:
            COLLECTED_DATE = ''.join((i.replace('Year and month in which the data were collected', '')).strip())
            TEXT = TEXT[TEXT.index(i)+1:]
    TEXT = [e for e in TEXT if e != ' ']
    TEXT = TEXT[TEXT.index('?  Copy ')+1:]
    DIC  = {'SCHOOL': SCHOOL, 'DISTRICT':DISTRICT, 'CDS_CODE':CDS_CODE, 'COLLECTED_DATE':COLLECTED_DATE,
            'TEXT_LST':TEXT}
    
    return DIC


In [ ]:
def SENT_CLEANER(lst):
    Final = [] 
    grades_ = []
    dates_ = []
    c = 0
    for s in lst:
      hype_num = re.findall(r'\d{1,2}-\d{1,2}', s)
      date     = re.findall('\d[0-9]\d\d', s)
      K_num    = re.findall(r'\sK-\d{1,2}', s)
      TK_num   = re.findall(r'\sTK-\d{1,2}', s)
      Dub_num  = re.findall(r'\s\d[0-9][^\d):]', s)
      sing_num = re.findall(r'\s\d{1}[^\d-]', s)
      if hype_num != [] or K_num != [] or TK_num != [] or Dub_num != [] or sing_num != []:
        c+=1
        grades_.append(c)
        if date != []:
          Final.append(s)
        else:
          Final.append(s)
      elif date != []:
          c+=1
          dates_.append(c)
          Final.append(s)
    temp = []
    for j in dates_:
        i = max([e for e in grades_ if j > e])
        s1 = Final[i-1]
        s2 = Final[j-1]
        temp.append(s1)
        temp.append(s2)
        new = s1+s2
        Final.append(new)
    for t in temp:
        Final.remove(t)
    FINAL = Furth_CLEANER(Final)
    return FINAL

In [ ]:
def Furth_CLEANER(lst):
    Final = [] 
    grades_ = []
    dates_ = []
    c = 0
    for s in lst:
      hype_num = re.findall(r'\d{1,2}-\d{1,2}', s)
      date     = re.findall('\d[0-9]\d\d', s)
      hype_date= re.findall(r'\s\d{4}-\d{4}\s', s)
      K_num    = re.findall(r'\sK-\d{1,2}', s)
      TK_num   = re.findall(r'\sTK-\d{1,2}', s)
      Dub_num  = re.findall(r'\s\d[0-9][^\d):]', s)
      sing_num = re.findall(r'\s\d{1}[^\d-]', s)
      words_alnum = ['1st', '2nd', '3rd', '4th', '5th', '6th', '7th', '8th', '9th', '10th']
      alpha_num  = [e for e in words_alnum if e in s]
      words_eng = ['First', 'Second', 'Third', 'Fourth', 'Fifth', 'Sixth', 'Seventh', 'Eighth', 'Ninth', 'Tenth']
      Eng_num  = [e for e in words_eng if e in s]
      if hype_num != [] or K_num != [] or TK_num != [] or Dub_num != [] or sing_num != [] or alpha_num != [] or Eng_num != []:
        c+=1
        grades_.append(c)
        if date != [] or hype_date != []:
          Final.append(s)
        else:
          Final.append(s)
      elif date != [] or hype_date != []:
          c+=1
          dates_.append(c)
          Final.append(s)
    temp = []
    for j in dates_:
        i = max([e for e in grades_ if j > e])
        s1 = Final[i-1]
        s2 = Final[j-1]
        temp.append(s1)
        temp.append(s2)
        new = s1+' '+s2
        Final.append(new)
    for t in temp:
        Final.remove(t)
    return Final

In [ ]:
def SENT_RET(path):
    DATA = BASIC_INFO(path)
    DICT = {}
    SUBJECTS = ['Reading/Language Arts', 'Mathematics', 'Science', 'History-Social Science', 'Visual and Performing Arts', 
            'Foreign Language', 'Health', 'Visual and Performing Arts', 'Science Laboratory Equipment', 'English Language Arts/Literacy']
    LST = [re.sub(' \s+ ', '', s) for s in DATA['TEXT_LST']]
    LST = [e for e in LST if e != '']
    word = 'English Language Arts/Literacy'
    ENG_LANG_ARTS_LITER_ = []
    for i in LST:
        try:
            SUBJECTS.remove('English Language Arts/Literacy')
        except:
            pass
        if word in i and 'Yes' in i:
          ENG_LANG_ARTS_LITER_.append(word)
          st_ = i.replace('Yes', '')
          st_ = st_.replace(' 0', '')
          st_ = st_.replace(' .0', '')
          st_ = st_.replace('%', '')
          ENG_LANG_ARTS_LITER_.append(" ".join((st_.split(word)[1]).split()))
          new_lst = LST[LST.index(i)+1:]
          for j in new_lst:
            if any(s in j for s in SUBJECTS) == False:
                ENG_LANG_ARTS_LITER_.append(j)
            else:
                break
          break
    
    try:
        ENG_LANG_ARTS_LITER = SENT_CLEANER([i for i in ENG_LANG_ARTS_LITER_[1:] if i != ''])
    except:
        ENG_LANG_ARTS_LITER = [i for i in ENG_LANG_ARTS_LITER_[1:] if i != '']
    if ENG_LANG_ARTS_LITER_ != []:
        DICT.update({ENG_LANG_ARTS_LITER_[0]:ENG_LANG_ARTS_LITER})

    word = 'Reading/Language Arts'
    READ_LANG_ARTS = []
    for i in LST:
        try:
            SUBJECTS.remove('Reading/Language Arts')
        except:
            pass
        if word in i and 'Yes' in i:
          READ_LANG_ARTS.append(word)
          st_ = i.replace('Yes', '')
          st_ = st_.replace(' 0', '')
          st_ = st_.replace(' .0', '')
          st_ = st_.replace('%', '')
          READ_LANG_ARTS.append(" ".join((st_.split(word)[1]).split()))
          new_lst = LST[LST.index(i)+1:]
          for j in new_lst:
            if any(s in j for s in SUBJECTS) == False:
                READ_LANG_ARTS.append(j)
            else:
                break
          break
    try:
        READ_LANG_ARTS_ = SENT_CLEANER([i for i in READ_LANG_ARTS[1:] if i != ''])
    except:
        READ_LANG_ARTS_ = [i for i in READ_LANG_ARTS[1:] if i != '']
    if READ_LANG_ARTS != []:
        DICT.update({READ_LANG_ARTS[0]:READ_LANG_ARTS_})

    word = 'Mathematics'
    MATH = []
    for i in LST:
        try:
            SUBJECTS.remove('Mathematics')
        except:
            pass
        if word in i and 'Yes' in i:
          MATH.append(word)
          st_ = i.replace('Yes', '')
          st_ = st_.replace(' 0', '')
          st_ = st_.replace(' .0', '')
          st_ = st_.replace('%', '')
          st_ = " ".join(st_.split())
          try:
              MATH.append(st_.split(' ', 1)[1])
          except:
              MATH.append('')
          new_lst = LST[LST.index(i)+1:]
          for j in new_lst:
            if any(s in j for s in SUBJECTS) == False:
                MATH.append(j)
            else:
                break
          break
    try:
        MATH_ = SENT_CLEANER([i for i in MATH[1:] if i != ''])
    except:
        MATH_ = [i for i in MATH[1:] if i != '']
    if MATH != []:
        DICT.update({MATH[0]:MATH_})

    word = 'Science'
    SCIENCE = []
    for i in LST:
        try:
            SUBJECTS.remove('Science')
        except:
            pass
        if word in i and 'Yes' in i:
          SCIENCE.append(word)
          st_ = i.replace('Yes', '')
          st_ = st_.replace(' 0', '')
          st_ = st_.replace(' .0', '')
          st_ = st_.replace('%', '')
          st_ = " ".join(st_.split())
          try:
              SCIENCE.append(st_.split(' ', 1)[1])
          except:
              SCIENCE.append('')
          new_lst = LST[LST.index(i)+1:]
          for j in new_lst:
            if any(s in j for s in SUBJECTS) == False:
                SCIENCE.append(j)
            else:
                break
          break
    try:
        SCIENCE_ = SENT_CLEANER([i for i in SCIENCE[1:] if i != ''])
    except:
        SCIENCE_ = [i for i in SCIENCE[1:] if i != '']
    if SCIENCE != []:
        DICT.update({SCIENCE[0]:SCIENCE_})

    word = 'History-Social Science'
    HIST_SOCIAL_SC = []
    for i in LST:
        try:
            SUBJECTS.remove('History-Social Science')
        except:
            pass
        if word in i and 'Yes' in i:
          HIST_SOCIAL_SC.append(word)
          st_ = i.replace('Yes', '')
          st_ = st_.replace(' 0', '')
          st_ = st_.replace(' .0', '')
          st_ = st_.replace('%', '')
          HIST_SOCIAL_SC.append(" ".join((st_.split(word)[1]).split()))
          new_lst = LST[LST.index(i)+1:]
          for j in new_lst:
            if any(s in j for s in SUBJECTS) == False:
                HIST_SOCIAL_SC.append(j)
            else:
                break
          break    
    try:
        HIST_SOCIAL_SC_ = SENT_CLEANER([i for i in HIST_SOCIAL_SC[1:] if i != ''])
    except:
        HIST_SOCIAL_SC_ = [i for i in HIST_SOCIAL_SC[1:] if i != '']   
    if HIST_SOCIAL_SC != []:
        DICT.update({HIST_SOCIAL_SC[0]:HIST_SOCIAL_SC_})

    return DATA['SCHOOL'], DATA['DISTRICT'], DATA['CDS_CODE'], DATA['COLLECTED_DATE'], DICT

In [ ]:
def GET_ROWS(path):
    Data = SENT_RET(path)
    OUT = []
    for k, v in zip(Data[4].keys(), Data[4].values()):
        for s in v:
            GRADE = []
            DATE  = []
            hype_num = re.findall(r'\d{1,2}-\d{1,2}', s)
            GRADE.append(hype_num)
            date     = re.findall('\d[0-9]\d\d', s)
            DATE.append(date)
            hype_date= re.findall(r'\s\d{4}-\d{4}\s', s)
            DATE.append(hype_date)
            K_num    = re.findall(r'\sK-\d{1,2}', s)
            GRADE.append(K_num)
            TK_num   = re.findall(r'\sTK-\d{1,2}', s)
            GRADE.append(TK_num)
            Dub_num  = re.findall(r'\s\d[0-9][^\d):]', s)
            GRADE.append(Dub_num)
            sing_num = re.findall(r'\s\d{1}[^\d-]', s)
            GRADE.append(sing_num)
            words_alnum = ['1st', '2nd', '3rd', '4th', '5th', '6th', '7th', '8th', '9th', '10th']
            alpha_num  = [e for e in words_alnum if e in s]
            GRADE.append(alpha_num)
            words_eng = ['First', 'Second', 'Third', 'Fourth', 'Fifth', 'Sixth', 'Seventh', 'Eighth', 'Ninth', 'Tenth']
            Eng_num  = [e for e in words_eng if e in s]
            GRADE.append(Eng_num)
            if hype_num != [] or K_num != [] or TK_num != [] or Dub_num != [] or sing_num != [] or alpha_num != [] or Eng_num != []:
                grade = [e for e in GRADE if e != []]
                if date != [] or hype_date != []:
                    Date= [e for e in DATE if e != []]
                    row = [Data[0], Data[1], Data[2], Data[3], k, s, grade[0][0], Date[0][0]]
                    OUT.append(row)
                else:
                    row = [Data[0], Data[1], Data[2], Data[3], k, s, grade[0][0], '']
                    OUT.append(row)
            elif date != [] or hype_date != []:
                Date= [e for e in DATE if e != []]
                if len(s.split()) > 3:
                    row = [Data[0], Data[1], Data[2], Data[3], k, s, '', Date[0][0]]
                    OUT.append(row)
    return OUT      

In [ ]:
def main(path):
    hdr = ['School', 'District', 'CDS Code', 'Collected Date', 'Subject', 'Curriculum', 'Grades', 'Adoption Year']
    pth = load_pdfs(path)
    with open('Output.csv', 'w', newline = '') as output_csv:
        # initialize rows writer
        csv_writer = csv.writer(output_csv)
        # write headers to the file
        csv_writer.writerow(hdr)
        for p in pth:
            Pth = path+'/'+p
            nest_lst = GET_ROWS(Pth)
            csv_writer.writerows(nest_lst)
    return          